In [ ]:
# install related library
!pip install transformers 
!pip install pythainlp
!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10766 sha256=3a43b0f78

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Dense, Flatten, Concatenate, Dropout, Dot, Activation, Reshape, Permute, Multiply
from keras import backend as K
import pandas as pd
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
# load dataset
data_path = '/content/drive/MyDrive/sharefile/residual_text/residual_text_data_e3.csv'
df = pd.read_csv(data_path, encoding='utf8')
df.head()

,text,valid,is_scene,has_num,turn_on_off,device,env,hour,minute
0,ปิดไฟห้องนั่งเล่น,1,0,0,0,0,ห้องนั่งเล่น,-1,-1
1,ปิดไฟห้องครัว,1,0,0,0,0,ห้องครัว,-1,-1
2,ปิดไฟห้องนอน,1,0,0,0,0,ห้องนอน,-1,-1
3,ปิดไฟห้องน้ำ,1,0,0,0,0,ห้องน้ำ,-1,-1
4,ปิดไฟห้องรับประทานอาหาร,1,0,0,0,0,ห้องรับประทานอาหาร,-1,-1


In [ ]:
df.describe()

,valid,is_scene,has_num,turn_on_off,device,hour,minute
count,914554.000000,914554.000000,914554.000000,914554.0,914554.0,914554.000000,914554.000000
mean,0.151195,0.075814,0.167951,0.5,0.0,11.241508,29.478856
std,0.358239,0.264700,0.373823,0.5,0.0,6.305869,17.330710
min,0.000000,0.000000,0.000000,0.0,0.0,-1.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.0,0.0,6.000000,14.000000
50%,0.000000,0.000000,0.000000,0.5,0.0,10.000000,29.000000
75%,0.000000,0.000000,0.000000,1.0,0.0,16.000000,44.000000
max,1.000000,1.000000,1.000000,1.0,0.0,23.000000,59.000000


In [ ]:
true_sentence = df[df['valid'] == 1]
false_sentence = df[df['valid'] == 0]

# sample invalid data for fast training
true_case_number  = true_sentence.shape[0]
random_false_sentence = false_sentence.sample(n=10000, random_state=42)


all_df = pd.concat([true_sentence, random_false_sentence])

#shuffle dataset
all_df = all_df.sample(frac=1, axis=0,random_state=42).reset_index(drop=True)

In [ ]:
all_df.head()

,text,valid,is_scene,has_num,turn_on_off,device,env,hour,minute
0,ปิดไฟช้างตอนแปดโมงหกสิบนาที,0,0,0,0,0,ช้าง,8,59
1,ปิดไฟห้องเล่นเกมเวลาสามทุ่มยี่สิบสามนาที,1,0,1,0,0,ห้องเล่นเกม,21,23
2,ปิดไฟห้องสมุดตอนแปดโมงเช้าห้านาที,1,0,1,0,0,ห้องสมุด,8,5
3,เปิดไฟซีน จีเวลาหกโมงหกนาที,1,1,1,1,0,ซีน จี,6,6
4,ปิดไฟห้องนอนตอนหกโมงห้านาที,1,0,1,0,0,ห้องนอน,6,5


In [ ]:
all_df.describe()

,valid,is_scene,has_num,turn_on_off,device,hour,minute
count,148276.000000,148276.000000,148276.000000,148276.000000,148276.0,148276.000000,148276.000000
mean,0.932558,0.415219,0.933718,0.499845,0.0,11.245481,29.487503
std,0.250786,0.492761,0.248775,0.500002,0.0,6.302594,17.324638
min,0.000000,0.000000,0.000000,0.000000,0.0,-1.000000,-1.000000
25%,1.000000,0.000000,1.000000,0.000000,0.0,6.000000,14.000000
50%,1.000000,0.000000,1.000000,0.000000,0.0,11.000000,30.000000
75%,1.000000,1.000000,1.000000,1.000000,0.0,16.000000,44.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,23.000000,59.000000


In [ ]:
# convert from dataframe to numpy
texts   =  all_df['text'].astype(str).values.tolist()
labels1 =  all_df['valid'].to_numpy()
labels2 = all_df['is_scene'].to_numpy()
labels3 = all_df['has_num'].to_numpy()
labels4 = all_df['turn_on_off'].to_numpy()
labels5 = all_df['device'].to_numpy()
labels6 = all_df['env'].astype(str).values.tolist()
labels7 = all_df['hour'].astype(str).values.tolist()
labels8 = all_df['minute'].astype(str).values.tolist()

texts[:5],labels1[:5],labels2[:5],labels3[:5],labels4[:5],labels5[:5],labels6[:5],labels7[:5],labels8[:5]

(['ปิดไฟช้างตอนแปดโมงหกสิบนาที',
  'ปิดไฟห้องเล่นเกมเวลาสามทุ่มยี่สิบสามนาที',
  'ปิดไฟห้องสมุดตอนแปดโมงเช้าห้านาที',
  'เปิดไฟซีน จีเวลาหกโมงหกนาที',
  'ปิดไฟห้องนอนตอนหกโมงห้านาที'],
 array([0, 1, 1, 1, 1]),
 array([0, 0, 0, 1, 0]),
 array([0, 1, 1, 1, 1]),
 array([0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0]),
 ['ช้าง', 'ห้องเล่นเกม', 'ห้องสมุด', 'ซีน จี', 'ห้องนอน'],
 ['8', '21', '8', '6', '6'],
 ['59', '23', '5', '6', '5'])

In [ ]:
from pythainlp import  word_tokenize

# split word using pythainlp lib
spilt_thai_text = [word_tokenize(x) for x in texts]
spilt_thai_text[:5]

[['ปิดไฟ', 'ช้าง', 'ตอน', 'แปด', 'โมง', 'หกสิบ', 'นาที'],
 ['ปิดไฟ',
  'ห้อง',
  'เล่น',
  'เกม',
  'เวลา',
  'สาม',
  'ทุ่ม',
  'ยี่สิบ',
  'สาม',
  'นาที'],
 ['ปิดไฟ', 'ห้องสมุด', 'ตอน', 'แปด', 'โมง', 'เช้า', 'ห้า', 'นาที'],
 ['เปิดไฟ', 'ซีน', ' ', 'จี', 'เวลา', 'หก', 'โมง', 'หก', 'นาที'],
 ['ปิดไฟ', 'ห้องนอน', 'ตอน', 'หก', 'โมง', 'ห้า', 'นาที']]

In [ ]:
# load the tokenizer and transformer model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base",max_length=60) 
transformer_model = TFAutoModel.from_pretrained("xlm-roberta-base") 

All model checkpoint layers were used when initializing TFXLMRobertaModel.

All the layers of TFXLMRobertaModel were initialized from the model checkpoint at xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [ ]:
# tokenize input
input_ids = tokenizer(spilt_thai_text, truncation=True, return_tensors="tf", padding=True,is_split_into_words=True)["input_ids"]

In [ ]:
# check highest length
max_seq_length = len(input_ids[0])
max_seq_length

32

In [ ]:
input_ids[95:100][:]

<tf.Tensor: shape=(5, 32), dtype=int32, numpy=
array([[     0,      6,  34031,  21128,      6,  73515,      6,  11771,
             6,  88239,  95702,      6, 168280,      6,  21041,   2824,
             6,  41165,      6, 168280,  79303,      2,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,      6,  34031,  21128,  82894,   2030,      6,  46924,
         45398,      6, 142575,  16963,  47287,      6,  21041,   2824,
             6,  13997,  79303,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,      6,  34031,  21128,  43538,  38048,  14662,  95702,
        220458,      6,  23278,      6,  34348,      6, 142575,  16963,
         47287,  79303,      2,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,  40001,  21128,  43538,  18379,  67261,  45398,      6,
        108590

In [ ]:
input_ids.shape

TensorShape([148276, 32])

In [ ]:
#create dict for mapping string to int and vice versa
num_class_label_6 =len(np.unique(labels6))
num_class_label_7 =len(np.unique(labels7))
num_class_label_8 =len(np.unique(labels8))


env_mapping = {x: i for i,x in enumerate(np.unique(labels6))}
env_decode = {i: x for i,x in enumerate(np.unique(labels6))}

hour_mapping = {x: i for i,x in enumerate(np.unique(labels7))}
hour_decode = {i: x for i,x in enumerate(np.unique(labels7))}


minute_mapping = {x: i for i,x in enumerate(np.unique(labels8))}
minute_decode = {i: x for i,x in enumerate(np.unique(labels8))}

tf_labels6 = [env_mapping[word] for word in labels6]
tf_labels7 = [hour_mapping[word] for word in labels7]
tf_labels8 = [minute_mapping[word] for word in labels8]

tf_labels6 = np.asarray(tf_labels6)
tf_labels7 = np.asarray(tf_labels7)
tf_labels8 = np.asarray(tf_labels8)

tf_labels6[:5],tf_labels7[:5],tf_labels8[:5],tf_labels6.shape,tf_labels7.shape,tf_labels8.shape

(array([19, 98, 97, 20, 92]),
 array([23, 15, 23, 21, 21]),
 array([56, 17, 46, 57, 46]),
 (148276,),
 (148276,),
 (148276,))

In [ ]:
# convert to one hot encoding 
tf_labels6 = tf.one_hot(tf_labels6,num_class_label_6)
tf_labels7 = tf.one_hot(tf_labels7,num_class_label_7)
tf_labels8 = tf.one_hot(tf_labels8,num_class_label_8)

In [ ]:
#define architecture

inputs = tf.keras.layers.Input(shape=(input_ids.shape[1],), dtype=tf.int32)
embedding_layer = transformer_model(inputs)[0]

flatten_layer = Flatten()(embedding_layer)
x1 = Dense(64, activation='relu')(flatten_layer)
x1 = Dense(32, activation='relu')(x1)
x1 = Dense(16, activation='relu')(x1)


x2 = Dense(64, activation='relu')(flatten_layer)
x2 = Dense(32, activation='relu')(x2)
x2 = Dense(16, activation='relu')(x2)

x3 = Dense(64, activation='relu')(flatten_layer)
x3 = Dense(32, activation='relu')(x3)
x3 = Dense(16, activation='relu')(x3)

x4 = Dense(64, activation='relu')(flatten_layer)
x4 = Dense(32, activation='relu')(x4)
x4 = Dense(16, activation='relu')(x4)


x5 = Dense(64, activation='relu')(flatten_layer)
x5 = Dense(32, activation='relu')(x5)
x5 = Dense(16, activation='relu')(x5)

x6 = Dense(512, activation='relu')(flatten_layer)
x6 = Dense(256, activation='relu')(x6)
x6 = Dense(128, activation='relu')(x6)

x7 = Dense(128, activation='relu')(flatten_layer)
x7 = Dense(64, activation='relu')(x7)
x7 = Dense(32, activation='relu')(x7)

x8 = Dense(256, activation='relu')(flatten_layer)
x8 = Dense(128, activation='relu')(x8)
x8 = Dense(64, activation='relu')(x8)


output_layer1 = Dense(1, activation='sigmoid', name='output1')(x1)
output_layer2 = Dense(1, activation='sigmoid', name='output2')(x2) 
output_layer3 = Dense(1, activation='sigmoid', name='output3')(x3) 
output_layer4 = Dense(1, activation='sigmoid', name='output4')(x4) 
output_layer5 = Dense(1, activation='sigmoid', name='output5')(x5) 
output_layer6 = Dense(num_class_label_6, activation='softmax', name='output6')(x6) 
output_layer7 = Dense(num_class_label_7, activation='softmax', name='output7')(x7) 
output_layer8 = Dense(num_class_label_8, activation='softmax', name='output8')(x8) 



In [ ]:
# train only last layer of transformer encoder
for i,layer in enumerate(transformer_model.roberta.encoder.layer[:-1]):
  transformer_model.roberta.encoder.layer[i].trainable = False

In [ ]:
# define the model  
model = Model(inputs=inputs , outputs=[output_layer1, output_layer2, output_layer3,output_layer4,output_layer5,output_layer6,output_layer7,output_layer8])

#set optimizer using adma
opt = keras.optimizers.Adam(learning_rate=3e-5) 

#define loss and metric
model.compile(loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], optimizer=opt,
              metrics=[
       tf.keras.metrics.BinaryAccuracy(),
        'categorical_accuracy'
    ])


In [ ]:
# load weight in case continue training
# model.load_weights("/content/drive/MyDrive/sharefile/residual_text/t1_m1.h5")

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 tfxlm_roberta_model (TFXLMRobe  TFBaseModelOutputWi  278043648  ['input_1[0][0]']                
 rtaModel)                      thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 32,                                                
                                768),                                                             
                                 pooler_output=(Non                                           

In [ ]:
history = model.fit(input_ids, [labels1,labels2,labels3,labels4,labels5,tf_labels6,tf_labels7,tf_labels8], epochs=3, verbose=1)

Epoch 1/3


4634/4634 [==============================] - 1041s 217ms/step - loss: 1.3997 - output1_loss: 0.0160 - output2_loss: 0.0101 - output3_loss: 0.0158 - output4_loss: 0.0077 - output5_loss: 0.0014 - output6_loss: 0.2398 - output7_loss: 0.2777 - output8_loss: 0.8311 - output1_binary_accuracy: 0.9954 - output1_categorical_accuracy: 1.0000 - output2_binary_accuracy: 0.9958 - output2_categorical_accuracy: 1.0000 - output3_binary_accuracy: 0.9950 - output3_categorical_accuracy: 1.0000 - output4_binary_accuracy: 0.9967 - output4_categorical_accuracy: 1.0000 - output5_binary_accuracy: 0.9999 - output5_categorical_accuracy: 1.0000 - output6_binary_accuracy: 0.9994 - output6_categorical_accuracy: 0.9503 - output7_binary_accuracy: 0.9958 - output7_categorical_accuracy: 0.9257 - output8_binary_accuracy: 0.9959 - output8_categorical_accuracy: 0.7989
Epoch 2/3
4634/4634 [==============================] - 1005s 217ms/step - loss: 0.0136 - output1_loss: 4.5955e-04 - output2_loss: 3.7824e-04 - output3_loss

In [ ]:
# model.save('s10_2.h5')

In [ ]:
model.evaluate(input_ids, [labels1,labels2,labels3,labels4,labels5,tf_labels6,tf_labels7,tf_labels8])

4634/4634 [==============================] - 411s 88ms/step - loss: 4.0948e-04 - output1_loss: 1.2711e-04 - output2_loss: 7.7445e-05 - output3_loss: 4.4014e-05 - output4_loss: 3.9280e-07 - output5_loss: 2.7512e-09 - output6_loss: 4.8535e-05 - output7_loss: 1.6597e-05 - output8_loss: 9.5383e-05 - output1_binary_accuracy: 1.0000 - output1_categorical_accuracy: 1.0000 - output2_binary_accuracy: 1.0000 - output2_categorical_accuracy: 1.0000 - output3_binary_accuracy: 1.0000 - output3_categorical_accuracy: 1.0000 - output4_binary_accuracy: 1.0000 - output4_categorical_accuracy: 1.0000 - output5_binary_accuracy: 1.0000 - output5_categorical_accuracy: 1.0000 - output6_binary_accuracy: 1.0000 - output6_categorical_accuracy: 1.0000 - output7_binary_accuracy: 1.0000 - output7_categorical_accuracy: 1.0000 - output8_binary_accuracy: 1.0000 - output8_categorical_accuracy: 1.0000


[0.00040948271634988487,
 0.00012711128511000425,
 7.744453614577651e-05,
 4.4014279410475865e-05,
 3.928045941847813e-07,
 2.7512314648703295e-09,
 4.8535286623518914e-05,
 1.6596568457316607e-05,
 9.538311860524118e-05,
 0.9999527931213379,
 1.0,
 0.9999729990959167,
 1.0,
 0.9999865293502808,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999998807907104,
 0.9999932646751404,
 1.0,
 1.0,
 0.9999996423721313,
 0.9999932646751404]

In [ ]:
# test the model on some new input sentences

test_texts = ['ปิดไฟซีน ซีเวลาตีสามสี่สิบเก้านาที','ปิดไฟห้องนอนตอนแปดโมงยี่สิบแปดนาที','ปิดไฟห้องรับประทานอาหารเวลาตีหนึ่งสี่สิบหกนาที','เปิดไฟห้องนอน','ปิดไฟช้างตอนแปดโมงหกสิบนาที']
spilt_thai_text = [word_tokenize(x) for x in test_texts]
new_input_ids = tokenizer(spilt_thai_text, padding=True, truncation=True, return_tensors="tf",is_split_into_words=True)["input_ids"]
test_padded_sequences = pad_sequences(new_input_ids, maxlen=max_seq_length,padding='post',truncating='post',value=1) #post pre
predicted_labels = model.predict(test_padded_sequences)

1/1 [==============================] - 3s 3s/step


In [ ]:
spilt_thai_text

[['ปิดไฟ', 'ซีน', ' ', 'ซี', 'เวลา', 'ตีสาม', 'สี่', 'สิบ', 'เก้า', 'นาที'],
 ['ปิดไฟ', 'ห้องนอน', 'ตอน', 'แปด', 'โมง', 'ยี่สิบ', 'แปด', 'นาที'],
 ['ปิดไฟ',
  'ห้อง',
  'รับประทาน',
  'อาหาร',
  'เวลา',
  'ตีหนึ่ง',
  'สี่',
  'สิบ',
  'หก',
  'นาที'],
 ['เปิดไฟ', 'ห้องนอน'],
 ['ปิดไฟ', 'ช้าง', 'ตอน', 'แปด', 'โมง', 'หกสิบ', 'นาที']]

In [ ]:
test_padded_sequences

array([[     0,      6,  34031,  21128,  82894,   2030,  82894,  45398,
             6,  43978,  34348,      6, 108138,      6, 142575,  26093,
        106439,  79303,      2,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,      6,  34031,  21128,      6, 136790,  95702,      6,
        108590,   2851,      6,  21041,   2824,  82556,  50693, 142575,
             6, 108590,   2851,  79303,      2,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,      6,  34031,  21128,  43538,      6, 174144,  66040,
         45398,      6,  43978,  13997,      6, 108138,      6, 142575,
             6, 206380,  79303,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1],
       [     0,  40001,  21128,      6, 136790,      2,      1,      1,
             1,      1,      1,      1,      1,      1,      

In [ ]:

for i in range(len(test_texts)):
  print(test_texts[i])
  valid = 1 if predicted_labels[0][i] > 0.5 else 0
  is_scene = 1 if predicted_labels[1][i] > 0.5 else 0
  has_num = 1 if predicted_labels[2][i] > 0.5 else 0
  print(f'is_valid : {valid}')
  print(f'is_scene : {is_scene}')
  print(f'has_num : {has_num}')

  turn = 1 if predicted_labels[3][i] > 0.5 else 0
  print(f'turn_on_off : {turn}')
  print(f'device : ไฟ')

  env_id = np.argmax(predicted_labels[5][i])
  env_label = env_decode[env_id]

  hour_id = np.argmax(predicted_labels[6][i])
  hour_label = hour_decode[hour_id]

  minute_id = np.argmax(predicted_labels[7][i])
  minute_label = minute_decode[minute_id]
  print(f'env : {env_label}')
  print(f'hour : {hour_label}')
  print(f'minute : {minute_label}')
  print('----')

ปิดไฟซีน ซีเวลาตีสามสี่สิบเก้านาที
is_valid : 1
is_scene : 1
has_num : 1
turn_on_off : 0
device : ไฟ
env : ซีน ซี
hour : 3
minute : 49
----
ปิดไฟห้องนอนตอนแปดโมงยี่สิบแปดนาที
is_valid : 1
is_scene : 0
has_num : 1
turn_on_off : 0
device : ไฟ
env : ห้องนอน
hour : 8
minute : 28
----
ปิดไฟห้องรับประทานอาหารเวลาตีหนึ่งสี่สิบหกนาที
is_valid : 1
is_scene : 0
has_num : 1
turn_on_off : 0
device : ไฟ
env : ห้องรับประทานอาหาร
hour : 1
minute : 46
----
เปิดไฟห้องนอน
is_valid : 1
is_scene : 0
has_num : 0
turn_on_off : 1
device : ไฟ
env : ห้องนอน
hour : -1
minute : -1
----
ปิดไฟช้างตอนแปดโมงหกสิบนาที
is_valid : 0
is_scene : 0
has_num : 0
turn_on_off : 0
device : ไฟ
env : ช้าง
hour : 8
minute : 59
----


In [ ]:
# !zip -r model.zip m1.h5

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive',force_remount=True)

In [ ]:
# !cp s10_2.h5 '/content/gdrive/MyDrive/sharefile/residual_text'